# **DVD Rentals: A PostgreSQL Analysis Project**
_by Raffy Alcazar, Data Analyst_

In this project, I explore a DVD rental business database using PostgreSQL, focusing on querying and analyzing key business metrics. The goal is to uncover insights into revenue generation, rental patterns, and inventory performance.

By leveraging SQL queries, I extract data from various tables, including payments, rentals, and films, to answer critical business questions. Key queries include exploring schema information, calculating monthly profits, identifying the top-grossing movies and categories, and analyzing the most rented films.

In [2]:
-- Get the list of available tables in our schema which is 'dvdrentals'
SELECT *
FROM information_schema.tables
WHERE table_schema = 'dvdrentals';

,table_catalog,table_schema,table_name,table_type,self_referencing_column_name,reference_generation,user_defined_type_catalog,user_defined_type_schema,user_defined_type_name,is_insertable_into,is_typed,commit_action
0,course_databases,dvdrentals,actor,BASE TABLE,NaN,NaN,NaN,NaN,NaN,YES,NO,NaN
1,course_databases,dvdrentals,address,BASE TABLE,NaN,NaN,NaN,NaN,NaN,YES,NO,NaN
2,course_databases,dvdrentals,category,BASE TABLE,NaN,NaN,NaN,NaN,NaN,YES,NO,NaN
3,course_databases,dvdrentals,film,BASE TABLE,NaN,NaN,NaN,NaN,NaN,YES,NO,NaN
4,course_databases,dvdrentals,customer,BASE TABLE,NaN,NaN,NaN,NaN,NaN,YES,NO,NaN
5,course_databases,dvdrentals,inventory,BASE TABLE,NaN,NaN,NaN,NaN,NaN,YES,NO,NaN
6,course_databases,dvdrentals,language,BASE TABLE,NaN,NaN,NaN,NaN,NaN,YES,NO,NaN
7,course_databases,dvdrentals,payment,BASE TABLE,NaN,NaN,NaN,NaN,NaN,YES,NO,NaN
8,course_databases,dvdrentals,film_actor,BASE TABLE,NaN,NaN,NaN,NaN,NaN,YES,NO,NaN
9,course_databases,dvdrentals,rental,BASE TABLE,NaN,NaN,NaN,NaN,NaN,YES,NO,NaN


In [3]:
-- Get the list of available columns per tables.
FROM information_schema.columns
WHERE table_schema = 'dvdrentals';

,table_catalog,table_schema,table_name,column_name,ordinal_position,column_default,is_nullable,data_type,character_maximum_length,character_octet_length,numeric_precision,numeric_precision_radix,numeric_scale,datetime_precision,interval_type,interval_precision,character_set_catalog,character_set_schema,character_set_name,collation_catalog,collation_schema,collation_name,domain_catalog,domain_schema,domain_name,udt_catalog,udt_schema,udt_name,scope_catalog,scope_schema,scope_name,maximum_cardinality,dtd_identifier,is_self_referencing,is_identity,identity_generation,identity_start,identity_increment,identity_maximum,identity_minimum,identity_cycle,is_generated,generation_expression,is_updatable
0,course_databases,dvdrentals,film_actor,actor_id,1,NaN,YES,integer,NaN,NaN,32.0,2.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,course_databases,pg_catalog,int4,NaN,NaN,NaN,NaN,1,NO,NO,NaN,NaN,NaN,NaN,NaN,NO,NEVER,NaN,YES
1,course_databases,dvdrentals,address,phone,6,NaN,YES,text,NaN,1.073742e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,course_databases,pg_catalog,text,NaN,NaN,NaN,NaN,6,NO,NO,NaN,NaN,NaN,NaN,NaN,NO,NEVER,NaN,YES
2,course_databases,dvdrentals,customer,first_name,2,NaN,YES,text,NaN,1.073742e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,course_databases,pg_catalog,text,NaN,NaN,NaN,NaN,2,NO,NO,NaN,NaN,NaN,NaN,NaN,NO,NEVER,NaN,YES
3,course_databases,dvdrentals,film,language_id,5,NaN,YES,integer,NaN,NaN,32.0,2.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,course_databases,pg_catalog,int4,NaN,NaN,NaN,NaN,5,NO,NO,NaN,NaN,NaN,NaN,NaN,NO,NEVER,NaN,YES
4,course_databases,dvdrentals,payment,payment_date,5,NaN,YES,timestamp with time zone,NaN,NaN,NaN,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,course_databases,pg_catalog,timestamptz,NaN,NaN,NaN,NaN,5,NO,NO,NaN,NaN,NaN,NaN,NaN,NO,NEVER,NaN,YES
5,course_databases,dvdrentals,actor,last_name,3,NaN,YES,text,NaN,1.073742e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,course_databases,pg_catalog,text,NaN,NaN,NaN,NaN,3,NO,NO,NaN,NaN,NaN,NaN,NaN,NO,NEVER,NaN,YES
6,course_databases,dvdrentals,customer,address_id,5,NaN,YES,integer,NaN,NaN,32.0,2.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,course_databases,pg_catalog,int4,NaN,NaN,NaN,NaN,5,NO,NO,NaN,NaN,NaN,NaN,NaN,NO,NEVER,NaN,YES
7,course_databases,dvdrentals,language,name,2,NaN,YES,text,NaN,1.073742e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,course_databases,pg_catalog,text,NaN,NaN,NaN,NaN,2,NO,NO,NaN,NaN,NaN,NaN,NaN,NO,NEVER,NaN,YES
8,course_databases,dvdrentals,film,special_features,11,NaN,YES,text,NaN,1.073742e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,course_databases,pg_catalog,text,NaN,NaN,NaN,NaN,11,NO,NO,NaN,NaN,NaN,NaN,NaN,NO,NEVER,NaN,YES
9,course_databases,dvdrentals,rental,rental_id,1,NaN,YES,integer,NaN,NaN,32.0,2.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,course_databases,pg_catalog,int4,NaN,NaN,NaN,NaN,1,NO,NO,NaN,NaN,NaN,NaN,NaN,NO,NEVER,NaN,YES


Based on the query above, this is the list of available table inside our schema.
![image](image.png)

Now, let's dive in and get some analysis/information in our tables.


In [7]:
-- First let's check what is the MIN and MAX of our date based on the 'payment' table.
SELECT
	MIN(p.payment_date) AS min_date,
	MAX(p.payment_date) AS max_date
FROM
	dvdrentals.payment AS p;

--Based on the result of our query below, we see that the oldest date is January 25, 2017 and the earliest is May 14, 2024.

,min_date,max_date
0,2017-01-25 02:21:56+00:00,2017-05-14 17:44:29+00:00


In [10]:
-- Now let's create a query that will get the total earnings per month to know which month have the highest profit.
SELECT
	EXTRACT(MONTH FROM p.payment_date) AS month,
	SUM(p.amount) AS total_profit
FROM
	dvdrentals.payment AS p
GROUP BY
	month
ORDER BY
	total_profit DESC;

--Based on the result of our query below, we see that the April(4) has the highest earnings from January to May 2017

,month,total_profit
0,4,27664.49
1,3,23886.56
2,2,9674.76
3,1,4781.55
4,5,1409.15


In [5]:
-- Top earnings based on the movie title
SELECT
	f.title AS movie_title,
	SUM(p.amount) AS total_profit
FROM
	dvdrentals.payment AS p
INNER JOIN dvdrentals.rental AS r
	ON p.rental_id = r.rental_id
INNER JOIN dvdrentals.inventory AS i
	ON r.inventory_id = i.inventory_id
INNER JOIN dvdrentals.film AS f
	ON i.film_id = f.film_id
INNER JOIN dvdrentals.category AS c
	ON f.film_id = c.film_id
GROUP BY
	movie_title
ORDER BY
	total_profit DESC;
--Telegraph Voyage has the highest earnings. Amounting to 231.73

,movie_title,total_profit
0,TELEGRAPH VOYAGE,231.73
1,WIFE TURN,223.69
2,ZORRO ARK,214.69
3,GOODFELLAS SALUTE,209.69
4,SATURDAY LAMBS,204.72
...,...,...
953,DUFFEL APOCALYPSE,6.93
954,YOUNG LANGUAGE,6.93
955,FREEDOM CLEOPATRA,5.95
956,TEXAS WATCH,5.94


In [2]:
--Most highest total earnings based on the category
SELECT
	c.category AS category,
	SUM(p.amount) AS total_profit
FROM
	dvdrentals.payment AS p
INNER JOIN dvdrentals.rental AS r
	ON p.rental_id = r.rental_id
INNER JOIN dvdrentals.inventory AS i
	ON r.inventory_id = i.inventory_id
INNER JOIN dvdrentals.film AS f
	ON i.film_id = f.film_id
INNER JOIN dvdrentals.category AS c
	ON f.film_id = c.film_id
GROUP BY
	category
ORDER BY
	total_profit DESC;
--Sports has the highest earnings based on the categories

,category,total_profit
0,Sports,5314.21
1,Sci-Fi,4756.98
2,Animation,4656.30
3,Drama,4587.39
4,Comedy,4383.58
5,Action,4375.85
6,New,4361.57
7,Games,4281.33
8,Foreign,4270.67
9,Family,4226.07


In [6]:
--most rented movie
SELECT
	f.title AS movie_title,
	COUNT(r.rental_id) AS num_rented
FROM
	dvdrentals.payment AS p
INNER JOIN dvdrentals.rental AS r
	ON p.rental_id = r.rental_id
INNER JOIN dvdrentals.inventory AS i
	ON r.inventory_id = i.inventory_id
INNER JOIN dvdrentals.film AS f
	ON i.film_id = f.film_id
GROUP BY
	movie_title
ORDER BY
	num_rented DESC;
--Most rented movie is 'BUCKET BROTHERHOOD'

,movie_title,num_rented
0,BUCKET BROTHERHOOD,34
1,ROCKETEER MOTHER,33
2,RIDGEMONT SUBMARINE,32
3,JUGGLER HARDLY,32
4,FORWARD TEMPLE,32
...,...,...
953,PRIVATE DROP,5
954,TRAFFIC HOBBIT,5
955,TRAIN BUNCH,4
956,MIXED DOORS,4


## **Here’s a summary of your queries and their results:**

**Summary of DVD Rentals SQL Queries**

1. Date Range of Payments:
- The earliest recorded payment date is January 25, 2017, while the latest is May 14, 2024. This gives us the full date range of rental transactions within the dataset.

2. Monthly Earnings (January–May 2017):
- April (month 4) stands out as the most profitable month in 2017, having the highest earnings compared to other months in the dataset.

3. Top-Grossing Movie:
- The movie Telegraph Voyage generated the highest total earnings, amounting to $231.73, making it the top-grossing film in the dataset.

4. Top-Earning Category:
- The Sports category leads in total earnings, surpassing other film categories.

5. Most Rented Movie:
- The most rented movie is Bucket Brotherhood, reflecting its popularity based on the number of rentals.